In [56]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV, LassoCV
import warnings

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

import numpy as np
import pandas as pd
import re

import xgboost as xgb

train = pd.read_csv('train.csv')
def extract_number(text):
    match = re.search(r'(\d+\.?\d*)', str(text))
    if match:
        return float(match.group())
    else:
        return 0.5  # if it doesn't have a number, it is 'half-bath'

train['baths'] = train['bathrooms_text'].apply(extract_number)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float) / 100

train['host_is_superhost'] = train['host_is_superhost'].astype('category')
train['host_location'] = train['host_location'].astype('category')
train['neighbourhood_cleansed'] = train['neighbourhood_cleansed'].astype('category')
train['property_type'] = train['property_type'].astype('category')
train['instant_bookable'] = train['instant_bookable'].astype('category')
train['amenities'] = train['amenities'].astype('category')
train['host_response_time'] = train['host_response_time'].astype('category')

In [62]:

warnings.filterwarnings("ignore")

y = train.loc[:, ['price']]
X = train.loc[:, ['host_location', #maybe
                  'host_response_time', #maybe
                  'host_acceptance_rate', 
                  'host_is_superhost', 
                  'neighbourhood_cleansed', 
                  'property_type', #'room_type'?
                  'accommodates', 
                  'baths', 
                  'beds',  #not bedrooms
                  'amenities', # or if it even has amenities, or pools vs hottub or pets
                  'maximum_maximum_nights', 
                  'maximum_nights',
                  'number_of_reviews', 
                  'number_of_reviews_ltm',  #check this
                  'instant_bookable'
]] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=307)


num_features = ['host_acceptance_rate', 
                'accommodates', 
                'baths', 
                'beds', 
                'maximum_maximum_nights', 
                'maximum_nights', 
                'number_of_reviews', 
                'number_of_reviews_ltm'
               ]

cat_features = ['host_response_time', 
                'host_location', 
                'host_is_superhost', 
                'neighbourhood_cleansed', 
                'property_type',
                'amenities', 
                'instant_bookable'
               ]


numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)



models = {
#     'Random Forest': RandomForestRegressor(), #Camilla
#     'Decision Tree Regressor': DecisionTreeRegressor(), # Daisy
    'SVR 10': SVR(C=10), 
    'SVR 1': SVR(C=1), 
    'SVR .1': SVR(C=.1), 
    'XGB - base': XGBRegressor(),
    'XGB - best params': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=7,subsample=.9),
#     'XGB - 1 depth 5': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=5,subsample=.9),
#     'XGB - 2 depth 9': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=9,subsample=.9),
#     'XGB - 3 sub .5': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=5,subsample=.5),
#     'XGB - 4 sub .3': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=5,subsample=.3),   
#     'XGB - 5 col .9': XGBRegressor(colsample_bytree = .9, learning_rate=.1, max_depth=5,subsample=.9),   
#     'XGB - 6 col .7': XGBRegressor(colsample_bytree = .7, learning_rate=.1, max_depth=5,subsample=.9),
#     'XGB - 7 - learn .01': XGBRegressor(colsample_bytree = 1, learning_rate=.01, max_depth=5,subsample=.5)   


}

# Create a pipeline for each model
pipelines = {}
for model_name, model in models.items():
    pipelines[model_name] = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Fit and evaluate each pipeline
results = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)  # Fit the pipeline
    y_pred = pipeline.predict(X_test)  # Make predictions
    mae = mean_absolute_error(y_test, y_pred)  # Calculate MAE
    results[model_name] = mae

# Print results
for model_name, mae in results.items():
    print(f"{model_name}: MAE = {mae}")



SVR 10: MAE = 154.93620542791422
SVR 1: MAE = 165.83814264073848
SVR .1: MAE = 180.12402156311964
XGB - base: MAE = 188.3676455633535
XGB - best params: MAE = 189.78980538575817


In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


warnings.filterwarnings("ignore")

y = train.loc[:, ['price']]
X = train.loc[:, ['host_location', #maybe
                  'host_response_time', #maybe
                  'host_acceptance_rate', 
                  'host_is_superhost', 
                  'neighbourhood_cleansed', 
                  'property_type', #'room_type'?
                  'accommodates', 
                  'baths', 
                  'beds',  #not bedrooms
                  'amenities', # or if it even has amenities, or pools vs hottub or pets
                  'maximum_maximum_nights', 
                  'maximum_nights',
                  'number_of_reviews', 
                  'number_of_reviews_ltm',  #check this
                  'instant_bookable'
]] 


num_features = ['host_acceptance_rate', 
                'accommodates', 
                'baths', 
                'beds', 
                'maximum_maximum_nights', 
                'maximum_nights', 
                'number_of_reviews', 
                'number_of_reviews_ltm'
               ]

cat_features = ['host_response_time', 
                'host_location', 
                'host_is_superhost', 
                'neighbourhood_cleansed', 
                'property_type',
                'amenities', 
                'instant_bookable'
               ]


numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)



xgb_regressor = XGBRegressor()

# Create the pipeline combining preprocessing and XGBoost model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor', xgb_regressor)
])

# Define the parameter grid to search
param_grid = {
    'xgb_regressor__learning_rate': [0.1, 0.01],
    'xgb_regressor__max_depth': [3, 5, 7],
    'xgb_regressor__subsample': [0.8, 0.9, 1.0],
    'xgb_regressor__colsample_bytree': [0.8, 0.9, 1.0]
}

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=307)

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error on test set:", mae)

Best parameters found: {'xgb_regressor__colsample_bytree': 1.0, 'xgb_regressor__learning_rate': 0.1, 'xgb_regressor__max_depth': 7, 'xgb_regressor__subsample': 0.9}
Mean Absolute Error on test set: 189.78980538575817


In [ ]:
# bytree 1, learning rate .1, max depth 7, subsample .9

In [63]:
num_features = ['host_acceptance_rate', 
                'accommodates', 
                'baths', 
                'beds', 
                'maximum_maximum_nights', 
                'maximum_nights', 
                'number_of_reviews', 
#                 'number_of_reviews_ltm'
               ]

cat_features = [#'host_response_time', 
#                 'host_location', 
                'host_is_superhost', 
                'neighbourhood_cleansed', 
                'property_type',
                'amenities', 
                'instant_bookable'
               ]

X = train.loc[:, [#'host_location', #maybe
                  #'host_response_time', #maybe
                  'host_acceptance_rate', 
                  'host_is_superhost', 
                  'neighbourhood_cleansed', 
                  'property_type', #'room_type'?
                  'accommodates', 
                  'baths', 
                  'beds',  #not bedrooms
                  'amenities', # or if it even has amenities, or pools vs hottub or pets
                  'maximum_maximum_nights', 
                  'maximum_nights',
                  'number_of_reviews', 
#                   'number_of_reviews_ltm',  #check this
                  'instant_bookable'
    ]] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=307)

numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)



models = {
#     'Random Forest': RandomForestRegressor(), #Camilla
#     'Decision Tree Regressor': DecisionTreeRegressor(), # Daisy
    'SVR 10': SVR(C=10), 
#     'SVR 1': SVR(C=1), 
#     'SVR .1': SVR(C=.1), 
    'XGB - base': XGBRegressor(),
    'XGB - best params': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=7,subsample=.9),  
}

# Create a pipeline for each model
pipelines = {}
for model_name, model in models.items():
    pipelines[model_name] = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Fit and evaluate each pipeline
results = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)  # Fit the pipeline
    y_pred = pipeline.predict(X_test)  # Make predictions
    mae = mean_absolute_error(y_test, y_pred)  # Calculate MAE
    results[model_name] = mae

# Print results
for model_name, mae in results.items():
    print(f"{model_name}: MAE = {mae}")

SVR 10: MAE = 155.98024167395775
XGB - base: MAE = 136.38005644678094
XGB - best params: MAE = 163.42094218602156


In [87]:
num_features = ['host_acceptance_rate', 
                'accommodates', 
                'baths', 
                'beds', 
#                 'maximum_maximum_nights', 
                'maximum_nights', 
                'number_of_reviews', 
#                 'number_of_reviews_ltm'
               ]

cat_features = [#'host_response_time', 
#                 'host_location', 
                'host_is_superhost', 
                'neighbourhood_cleansed', 
                'property_type',
                'amenities', 
                'instant_bookable'
               ]



# What helps most to least in the model
X = train.loc[:, [
                  # USE FOR SURE
                  'amenities', # NEEEEED THIS ONE
                  'property_type', # 9 points
                  'baths', # 9 points
                  'host_acceptance_rate', # 7 points
                  'beds',  # 6 points
                  'neighbourhood_cleansed',  # 5 points
                  'accommodates', # 4 points
                  'host_is_superhost', # 1 point
                  'instant_bookable', # .3 points
                  'maximum_nights', # .3 points
                  'number_of_reviews', # .3 points

    
                    # SHOULD TAKE OUT
#                   'number_of_reviews_ltm',  # LARGE negative effect
#                   'host_location', # medium negative effect
#                   'host_response_time', # small negative effect
#                   'maximum_maximum_nights', # - .2

    ]] 





# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=307)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3071)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30714)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=307149)




numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)



models = {
#     'Random Forest': RandomForestRegressor(), #Camilla
#     'Decision Tree Regressor': DecisionTreeRegressor(), # Daisy
    'SVR 10': SVR(C=10), 
    'XGB - base': XGBRegressor(),
    'XGB - best params': XGBRegressor(colsample_bytree = 1, learning_rate=.1, max_depth=7,subsample=.9),  
}

# Create a pipeline for each model
pipelines = {}
for model_name, model in models.items():
    pipelines[model_name] = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Fit and evaluate each pipeline
results = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)  # Fit the pipeline
    y_pred = pipeline.predict(X_test)  # Make predictions
    mae = mean_absolute_error(y_test, y_pred)  # Calculate MAE
    results[model_name] = mae

# Print results
for model_name, mae in results.items():
    print(f"{model_name}: MAE = {mae}")

SVR 10: MAE = 125.91300706894509
XGB - base: MAE = 115.85598036146398
XGB - best params: MAE = 142.1385458638742


In [ ]:
# best 136.11702607524765

# leaderboard - individual: 128.7

# after trying with xgboost:

In [89]:
train = pd.read_csv('train.csv')
train1 = pd.read_csv('test.csv')

def extract_number(text):
    match = re.search(r'(\d+\.?\d*)', str(text))
    if match:
        return float(match.group())
    else:
        return 0.5  # if it doesn't have a number, it is 'half-bath'

train['baths'] = train['bathrooms_text'].apply(extract_number)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float) / 100
train['host_is_superhost'] = train['host_is_superhost'].astype('category')
train['host_location'] = train['host_location'].astype('category')
train['neighbourhood_cleansed'] = train['neighbourhood_cleansed'].astype('category')
train['property_type'] = train['property_type'].astype('category')
train['instant_bookable'] = train['instant_bookable'].astype('category')
train['amenities'] = train['amenities'].astype('category')
train['host_response_time'] = train['host_response_time'].astype('category')


#same for test
train1['baths'] = train1['bathrooms_text'].apply(extract_number)
train1['host_acceptance_rate'] = train1['host_acceptance_rate'].str.rstrip('%').astype(float) / 100
train1['host_is_superhost'] = train1['host_is_superhost'].astype('category')
train1['host_location'] = train1['host_location'].astype('category')
train1['neighbourhood_cleansed'] = train1['neighbourhood_cleansed'].astype('category')
train1['property_type'] = train1['property_type'].astype('category')
train1['instant_bookable'] = train1['instant_bookable'].astype('category')
train1['amenities'] = train1['amenities'].astype('category')
train1['host_response_time'] = train1['host_response_time'].astype('category')


numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)


features = train.loc[:, [
                  # USE FOR SURE
                  'amenities', # NEEEEED THIS ONE
                  'property_type', # 9 points
                  'baths', # 9 points
                  'host_acceptance_rate', # 7 points
                  'beds',  # 6 points
                  'neighbourhood_cleansed',  # 5 points
                  'accommodates', # 4 points
                  'host_is_superhost', # 1 point
                  'instant_bookable', # .3 points
                  'maximum_nights', # .3 points
                  'number_of_reviews', # .3 points
    ]] 

y = train.loc[:, ['price']]

pipe = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("model", XGBRegressor())])


pipe.fit(features, y)
new_y = pipe.predict(train1)
new_y

final = train1.copy()

columns_to_drop = train1.columns.difference(['Id'])
final = final.drop(columns=columns_to_drop)

final['price'] = new_y
final.to_csv('final_predictions2.csv', index=False)